Importando as bibliotecas

In [12]:
import pandas as pd
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait

Definindo as variáveis

In [13]:
df = pd.read_excel("CREA-MS.xlsx", sheet_name=0, nrows=20)

In [14]:
df = df.drop(columns=['A','B', 'RT'], axis=1)

In [15]:
df2 = df[df['Situação'] == 'ATIVA']

In [16]:
df2.shape

(16, 3)

In [17]:
df2.head()

,Empresa,Registro,Situação
0,A7 AVIAÇÃO AGRÍCOLA,MS374,ATIVA
1,IMAGETECH TECNOLOGIA EM INFORMÁTICA LTDA,MS7507,ATIVA
2,03 IRMÃOS PRESTADORA DE SERVIÇOS,MS18358,ATIVA
3,1 PRA 2 ARQUITETURA E ENGENHARIA,MS19107,ATIVA
5,1A SERVICE,MS20561,ATIVA


In [18]:
url = 'http://cnpj.info/'

busca = 'A7 AVIACAO AGRICOLA'

cnpj = []
nome = []
fantasia = []
estado = []
limite_requisicoes = 0

Inicializando o navegador Firefox

In [19]:
browser = Firefox()
browser.get(url)

Insere nome da empresa no navegador, e faz a busca no site, retornando página nova

EXEMPLO DE CÓDIGO HTML A SER BUSCADO E EXTRAÍDO

<div class=hdr itemscope itemtype="http://schema.org/WebSite">
<meta itemprop="url" content="http://cnpj.info/">
<form method=post action="/busca" itemprop="potentialAction" itemscope itemtype="http://schema.org/SearchAction">
<meta itemprop="target" content="http://cnpj.info/{q}">
<h3>Pesquisa: <input type=text name=q placeholder="Nome ou CNPJ ou telefone" size=30 itemprop="query-input" required> <input type=submit value="Buscar" title="Search/Lookup"></h3></form>
</div>

In [20]:
divclass = browser.find_element_by_class_name('hdr')
element = divclass.find_element_by_tag_name('h3')
insert = element.find_element_by_name('q')
click = element.find_element_by_xpath("//input[@title='Search/Lookup']")

# Insere o texto com o nome da empresa na caixa de busca do navegador
insert.send_keys(busca)

# Clica no botão Buscar, outra opção é teclar enter no campo de busca = insert.send_keys(Keys.RETURN)
click.click()

# Atualiza o browser para a pagina atual, a pagina nova carregada após a busca
browser.current_window_handle

'17'

Espera pela lista de empresas buscadas carregar na página
Após espera anterior, carrega as empresas encontradas em cada <li> numa lista

In [10]:
def esperar_lista(browser):
    elements = browser.find_elements_by_tag_name('li')
    print("Tentando encontrar lista carregada")
    return bool(elements)

wdw = WebDriverWait(browser, 10)
wdw.until(esperar_lista, "Não carregou nenhuma lista")
list = browser.find_elements_by_tag_name('li')

Tentando encontrar lista carregada


Verifica se a pagina carregana não é página de erro de excesso de requisições


In [21]:
text_site = 'Too many requests per address per day... try to access this site later.'
text_many = browser.find_element_by_tag_name('h1')
print(text_many.text)
if text_many.text == text_site:
    print("Muitas requisições feitas hoje, bora mudar o IP do proxy")
    return 5

Too many requests per address per day... try to access this site later.


SyntaxError: 'return' outside function (<ipython-input-21-cc3810afb226>, line 6)

Imprime os elementos de cada <li> extraida

In [12]:
i = 0
while i < len(list):
    print(list[i].text +'\n')
    i = i+1

CNPJ: 03.154.507/0003-50 | 03154507000350 [ FILIAL ] - BAIXADA desde 2004-01-23
Nome: A7 Aviacao Agricola Eireli
Fantasia nome: Teruel Aviacao Agricola LTDA
Localização: Zona Rural, Campo Novo do Parecis - MT

CNPJ: 03.154.507/0005-11 | 03154507000511 [ FILIAL ] - BAIXADA desde 2018-07-30
Nome: A7 Aviacao Agricola Eireli
Fantasia nome: Teruel Luft Aviacao Agricola
Localização: Centro, Sao Gabriel do Oeste - MS

CNPJ: 03.154.507/0006-00 | 03154507000600 [ FILIAL ] - ATIVA desde 2015-07-23
Nome: A7 Aviacao Agricola Eireli
Fantasia nome: Teruel Luft Aviacao Agricola
Localização: Zona Rural, Campo Grande - MS

CNPJ: 03.154.507/0001-98 | 03154507000198 [ MATRIZ ] - ATIVA desde 2005-11-03
Nome: A7 Aviacao Agricola Eireli
Fantasia nome: A7 Aviacao Agricola
Localização: Centro, Chapadao do Sul - MS

CNPJ: 03.154.507/0002-79 | 03154507000279 [ FILIAL ] - BAIXADA desde 1995-07-04
Nome: A7 Aviacao Agricola Eireli
Localização: Sao Gabriel Dooeste, Camapua - MS

CNPJ: 03.154.507/0004-30 | 031545070

Extrai os valores CNPJ, Nome e Fantasia de cada <li> encontrado e adicionam a listas

CÓDIGO HTML A SER BUSCADO E EXTRAÍDO

<ul><li>CNPJ: <a href=/27316450000144>27.316.450/0001-44</a> | 27316450000144 [&nbsp;MATRIZ&nbsp;] - ATIVA&nbsp;desde&nbsp;2017-03-16<br>Nome: <a href=/Deniel-Willian-P-de-Castro-03-Irmaos-Prestadora-de-Servicos>Deniel Willian P de Castro</a><br>Fantasia nome: <a href=/Deniel-Willian-P-de-Castro-03-Irmaos-Prestadora-de-Servicos>03 Irmaos Prestadora de Servicos</a><br>Localização: Zona de Expansao Urbana, Terenos&nbsp;-&nbsp;MS
<li>CNPJ: <a href=/11040568000152>11.040.568/0001-52</a> | 11040568000152 [&nbsp;MATRIZ&nbsp;] - INAPTA&nbsp;desde&nbsp;2021-02-23<br>Nome: <a href=/J-Lomas-de-Souza-2-Irmaos-Prestadora-de-Servicos>J Lomas de Souza</a><br>Fantasia nome: <a href=/J-Lomas-de-Souza-2-Irmaos-Prestadora-de-Servicos>2 Irmaos Prestadora de Servicos</a><br>Localização: Mauazinho, Manaus&nbsp;-&nbsp;AM
<li>CNPJ: <a href=/27516078000110>27.516.078/0001-10</a> | 27516078000110 [&nbsp;MATRIZ&nbsp;] - INAPTA&nbsp;desde&nbsp;2021-02-23<br>Nome: <a href=/Claudio-Rodrigues-dos-Santos-Prestadora-de-Servicos-03-Irmaos>Claudio Rodrigues dos Santos</a><br>Fantasia nome: <a href=/Claudio-Rodrigues-dos-Santos-Prestadora-de-Servicos-03-Irmaos>Prestadora de Servicos 03 Irmaos</a><br>Localização: Centro, Porto Grande&nbsp;-&nbsp;AP
</ul>

In [13]:
inside = list[0].find_elements_by_tag_name('a')
inside

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="e58e128b-e1d7-48ec-b0d6-60bf4efe2db7", element="b7ef3609-65ff-459c-a179-b3444740bf48")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="e58e128b-e1d7-48ec-b0d6-60bf4efe2db7", element="99d92a92-7152-4075-92ed-589f9de7159d")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="e58e128b-e1d7-48ec-b0d6-60bf4efe2db7", element="e9fe4aac-b4df-4c16-af51-b2590f63121d")>]

In [30]:
inside[2].text

'Teruel Aviacao Agricola LTDA'

In [14]:
i = 0
while i < len(list):
    inside = list[i].find_elements_by_tag_name('a')
    
    try:
        cnpj.append(inside[0].text)
    except IndexError:
        cnpj.append('NaN')

    try:
        nome.append(inside[1].text)
    except IndexError:
        nome.append('NaN')    
    
    try:
        fantasia.append(inside[2].text)
    except IndexError:
        fantasia.append('NaN')
    
    # Extrai o Estado do país contído no texto
    list2 = list[i].text
    list3 = list2.splitlines()
    trash, loc = list3[-1].split(':')
    cidade, state = loc.split('-')
    state = state.strip()
    estado.append(state)
    
    i = i+1

In [15]:
print(cnpj)
print(nome)
print(fantasia)
print(estado)

['03.154.507/0003-50', '03.154.507/0005-11', '03.154.507/0006-00', '03.154.507/0001-98', '03.154.507/0002-79', '03.154.507/0004-30']
['A7 Aviacao Agricola Eireli', 'A7 Aviacao Agricola Eireli', 'A7 Aviacao Agricola Eireli', 'A7 Aviacao Agricola Eireli', 'A7 Aviacao Agricola Eireli', 'A7 Aviacao Agricola Eireli']
['Teruel Aviacao Agricola LTDA', 'Teruel Luft Aviacao Agricola', 'Teruel Luft Aviacao Agricola', 'A7 Aviacao Agricola', 'NaN', 'Agroatel LTDA']
['MT', 'MS', 'MS', 'MS', 'MS', 'MS']


In [16]:
# len(cnpj)
# len(nome)
# len(fantasia)
len(estado)

6

Convertes as listas extraídas no formato dataframe para melhor manipulação dos dados

In [17]:
dicio = {"CNPJ":cnpj, "Nome":nome, "Fantasia": fantasia, "Estado": estado}

In [18]:
df= pd.DataFrame(dicio)
df.head()

,CNPJ,Nome,Fantasia,Estado
0,03.154.507/0003-50,A7 Aviacao Agricola Eireli,Teruel Aviacao Agricola LTDA,MT
1,03.154.507/0005-11,A7 Aviacao Agricola Eireli,Teruel Luft Aviacao Agricola,MS
2,03.154.507/0006-00,A7 Aviacao Agricola Eireli,Teruel Luft Aviacao Agricola,MS
3,03.154.507/0001-98,A7 Aviacao Agricola Eireli,A7 Aviacao Agricola,MS
4,03.154.507/0002-79,A7 Aviacao Agricola Eireli,NaN,MS


In [19]:
df.shape

(6, 4)

In [46]:
dfnew = df[df['Estado'] == 'PR']

In [47]:
dfnew.head()

,CNPJ,Nome,Fantasia,Estado


In [48]:
len(dfnew)

0

In [22]:
listacnpj = dfnew.iloc[0]['CNPJ']

In [23]:
listacnpj

'03.154.507/0005-11'

In [24]:
type(listacnpj)

str

In [35]:
abc = 'Localização: Centro, Porto Grande - AP'
abc[-2:]

'AP'

In [38]:
caralho = """CNPJ: 03.154.507/0005-11 | 03154507000511 [ FILIAL ] - BAIXADA desde 2018-07-30
Nome: A7 Aviacao Agricola Eireli
Fantasia nome: Teruel Luft Aviacao Agricola
Localização: Centro, Sao Gabriel do Oeste - MS"""

In [40]:
list3 = caralho.splitlines()
list3

['CNPJ: 03.154.507/0005-11 | 03154507000511 [ FILIAL ] - BAIXADA desde 2018-07-30',
 'Nome: A7 Aviacao Agricola Eireli',
 'Fantasia nome: Teruel Luft Aviacao Agricola',
 'Localização: Centro, Sao Gabriel do Oeste - MS']

In [45]:
state = list3[-1][-2:]
state

'MS'

In [44]:
state = state.strip()

In [11]:
def funcao():
    return 5

for i in range(10):
    print(i)
    
    if funcao() == 5:
        print("Li dentro")
        break


0
Li dentro
